In [1]:
spark.conf.set("fs.azure.account.key.recruitstg.dfs.core.windows.net","iQjxUD39eP0oA2OnPD0LuQxsp8+adeWQfJcoOEeRuOrGmnxk+DPV0/CshK3llLfIgIEcwTbhZ0J6wwOscBAlXQ==")
#spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.functions import col, expr, when
from pyspark.sql import functions as F

In [3]:
# Read the Recriutment master file  as a dataframe.
check1=0
try:
  Rec = spark.read.csv("abfss://staging@recruitstg.dfs.core.windows.net/Recruitment_Master/recruitment.csv",header = True,inferSchema = True)
  if Rec.count()>0:
    check1=1
  else:
    print("Empty File")
except Exception as e:
  print(e)
  print("No File detected")
if check1==1:
  print(Rec.count())
  #Filtering unwanted rows with no values. 
  Rec=Rec.filter(Rec["Role"].isNotNull() & Rec.Status.isNull())
  #Rec=spark.createDataFrame(RecFiltered,schema=Rec.schema)
#   print(Rec.count())
#   Rec.show(1)

163
158
+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+
Request_Id| Role|Experience|Client|Total_Requisition|Project| Opened_On|Recruiter|Priority| Location|On_Hold_Date|Resumed_Date|Cancelled_Date|Skills|Role_Category|Status|Remarks|
+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+
 1|Solution Architec...| Mid level|Timken| 4| Timken|2019-02-01 00:00:00| Raja| P1|Bangalore| null| null| null| null| developer| null| null|
+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+
only showing top 1 row

In [4]:
# Read the employee master file as dataframe.
check2=0
try:
  Emp = spark.read.csv("abfss://staging@recruitstg.dfs.core.windows.net/Employee_Master/employee.csv",header = True,inferSchema = True)
  if Emp.count()>0:
    check2=1
  else:
    print("Empty File")
except:
  print("No File detected")
if check2==1:
  print(Emp.count())
  #Filtering unwanted rows with no values. 
  Emp=Emp.filter(Emp["Request_Id"].isNotNull())
  #Emp=spark.createDataFrame(EmpFiltered,schema=Emp.schema)
#   print(Emp.count())
#   Emp.show(1)

474
474
+------+----+------------+----------+-------------------+------+-----------+---------+
Emp_Id|Name|Candidate_Id|Request_Id| Joined_Date|Salary|Last_Salary|Hike_Pert|
+------+----+------------+----------+-------------------+------+-----------+---------+
 1|null| 5| 3|2019-02-12 00:00:00|400000| 285714| 40.0|
+------+----+------------+----------+-------------------+------+-----------+---------+
only showing top 1 row

In [5]:
#Read the candidate data file as dataframe.
check3=0
try:
  CandidateDF = spark.read.csv("abfss://staging@recruitstg.dfs.core.windows.net/Candidate_Master/candidate.csv",header = True,inferSchema = True)
  if CandidateDF.count()>0:
    check3=1
  else:
    print("Empty File")
except:
  print("No File detected")
if check3==1:
  #Filtering unwanted rows with no values.
  print(CandidateDF.count())
  CandidateDF=CandidateDF.filter(CandidateDF["Gender"].isNotNull())
  #CandidateDF=spark.createDataFrame(CandidateDFFiltered,schema=CandidateDF.schema)
  #print(CandidateDF.count())
  #CandidateDF.show(1)

1265
1265
+------------+----+------+--------+------------+---------------+-------------------+-----------+-------------------+--------+-------+
Candidate_Id|Name|Gender|Email_Id|Phone_Number| Source| Career_Started_At|Break_Taken|Relevant_Experience| Status|Offered|
+------------+----+------+--------+------------+---------------+-------------------+-----------+-------------------+--------+-------+
 1|null| M| null| null|company website|2018-01-04 00:00:00| No| 1.92|rejected| null|
+------------+----+------+--------+------------+---------------+-------------------+-----------+-------------------+--------+-------+
only showing top 1 row

In [6]:
#Reading the refferal file as dataframe.
check4=0
try:
  ReferralDF = spark.read.csv("abfss://staging@recruitstg.dfs.core.windows.net/Referral_Master/referral.csv",header = True,inferSchema = True)
  if ReferralDF.count()>0:
    check4=1
  else:
    print("Empty File")
except:
  print("No File detected")
if check4==1:
  print(ReferralDF.count())
  #Filtering unwanted rows with no values. 
  ReferralDF=ReferralDF.filter(ReferralDF["Candidate_Id"].isNotNull())
  #ReferralDF=spark.createDataFrame(ReferralDFFiltered,schema=ReferralDF.schema)
  #print(ReferralDF.count())
  #ReferralDF.show(1)

204
198
+-----------+------------+------+-------------+------+
Referral_Id|Candidate_Id|Emp_Id|Referral_Date|Status|
+-----------+------------+------+-------------+------+
 1| 16| 4| null| null|
+-----------+------------+------+-------------+------+
only showing top 1 row

In [7]:
#Reading the interview file as dataframe.
check5=0
try:
  interviewDF = spark.read.csv("abfss://staging@recruitstg.dfs.core.windows.net/Interview_Master/interview.csv",header = True,inferSchema = True)
  if interviewDF.count()>0:
    check5=1
  else:
    print("Empty File")
except:
  print("No File detected")
if check5==1:
  print(interviewDF.count())
  #Filtering unwanted rows with no values. 
  interviewDF=interviewDF.filter(interviewDF["Candidate_Id"].isNotNull())
  #interviewDF=spark.createDataFrame(interviewDFFiltered,schema=interviewDF.schema)
  #print(interviewDF.count())
  #interviewDF.show(1)
  

57
57
+------------+------------+--------------+---------+------+--------------+-------+-------+
Interview_Id|Candidate_Id|Interviewer_Id| Round|Result|Interview_Date| Status|Remarks|
+------------+------------+--------------+---------+------+--------------+-------+-------+
 1| 1261| null|paper&pen|passed| null|ongoing| null|
+------------+------------+--------------+---------+------+--------------+-------+-------+
only showing top 1 row

In [8]:
# Latest Status update for candidate. 
if check1==1&check2==1&check3==1&check4==1&check5==1:
  interviewDF2 = interviewDF.groupBy("Candidate_Id").agg(max("Interview_Id").alias("Interview_Id_L"))
  interviewDF2 = interviewDF.join(interviewDF2,[interviewDF2["Interview_Id_L"] == interviewDF["Interview_Id"]]).drop(interviewDF2["Interview_Id_L"]).drop(interviewDF2["Candidate_Id"])
  CandidateDF=CandidateDF.join(interviewDF2,[CandidateDF["Candidate_Id"]==interviewDF2["Candidate_Id"]]).drop(CandidateDF.Status).drop(CandidateDF.Candidate_Id).drop(interviewDF2.Interview_Id).drop(interviewDF2.Interviewer_Id).drop(interviewDF2.Interview_Date)
  #interviewDF2.show()

+------------+--------------+-----------------+-------+--------------+--------+-------+------------+
Interview_Id|Interviewer_Id| Round| Result|Interview_Date| Status|Remarks|Candidate_Id|
+------------+--------------+-----------------+-------+--------------+--------+-------+------------+
 4| null| HR| passed| null|selected| null| 1261|
 7| null| HR|ongoing| null| ongoing| null| 1258|
 10| null| HR| passed| null|selected| null| 1255|
 13| null| HR| passed| null|selected| null| 1251|
 15| null|technical round-2| passed| null| ongoing| null| 1246|
 19| null| HR| passed| null|selected| null| 1265|
 23| null| HR| passed| null|selected| null| 1259|
 26| null| HR| passed| null|selected| null| 1256|
 29| null| HR| passed| null|selected| null| 1257|
 31| null|technical round-2| failed| null|rejected| null| 1247|
 34| null| HR| passed| null|selected| null| 1254|
 35| null|technical round-1| passed| null| ongoing| null| 1264|
 38| null|technical round-2| failed| null|rejected| null| 1252|
 41| null| HR| passed| null|selected| null| 1263|
 44| null| HR| passed| null|selected| null| 1262|
 45| null|technical round-1| passed| null| ongoing| null| 1250|
 49| null| HR| passed| null|selected| null| 1260|
 51| null|technical round-2| passed| null| ongoing| null| 1253|
 53| null|technical round-1| passed| null| ongoing| null| 1248|
 55| null|technical round-2| failed| null|rejected| null| 1249|
+------------+--------------+-----------------+-------+--------------+--------+-------+------------+
only showing top 20 rows

In [9]:
#ReferralDF.show()

+-----------+------------+------+-------------+------+
Referral_Id|Candidate_Id|Emp_Id|Referral_Date|Status|
+-----------+------------+------+-------------+------+
 1| 16| 4| null| null|
 2| 18| 6| null| null|
 3| 31| 1| null| null|
 4| 54| 6| null| null|
 5| 92| 24| null| null|
 6| 108| 8| null| null|
 7| 112| 14| null| null|
 8| 125| 48| null| null|
 9| 142| 38| null| null|
 10| 215| 6| null| null|
 11| 238| 25| null| null|
 12| 264| 82| null| null|
 13| 276| 21| null| null|
 14| 280| 48| null| null|
 15| 345| 11| null| null|
 16| 357| 41| null| null|
 17| 383| 138| null| null|
 18| 421| 3| null| null|
 19| 429| 117| null| null|
 20| 445| 157| null| null|
+-----------+------------+------+-------------+------+
only showing top 20 rows

In [10]:
#Latest Status Update for referrer.
if check1==1&check2==1&check3==1&check4==1&check5==1:
  ReferralDF=ReferralDF.join(interviewDF2,[ReferralDF["Candidate_Id"]==interviewDF2["Candidate_Id"]]).drop(ReferralDF.Status).drop(ReferralDF.Candidate_Id)[['Referral_Id','Candidate_Id','Emp_Id','Referral_Date','Status']]
  #ReferralDF.show()

+-----------+------------+------+-------------+--------+
Referral_Id|Candidate_Id|Emp_Id|Referral_Date| Status|
+-----------+------------+------+-------------+--------+
 196| 1261| 459| null|selected|
 195| 1258| 458| null| ongoing|
 192| 1251| 455| null|selected|
 198| 1265| 460| null|selected|
 194| 1254| 457| null|selected|
 197| 1264| 460| null| ongoing|
 191| 1250| 454| null| ongoing|
 193| 1253| 456| null| ongoing|
 190| 1248| 453| null| ongoing|
 189| 1266| 47| null|selected|
+-----------+------------+------+-------------+--------+

In [11]:
# Number of filled positions/ Joined count.
if check1==1&check2==1&check3==1&check4==1&check5==1:
  joined_df = Emp.groupBy("Request_Id").agg(count("Emp_Id").alias("joined_cnt"))
  #joined_df.show(3)

+----------+----------+
Request_Id|joined_cnt|
+----------+----------+
 148| 4|
 31| 1|
 85| 3|
+----------+----------+
only showing top 3 rows

In [12]:
# Add the joined count column to the recruitment dataframe.
if check1==1&check2==1&check3==1&check4==1&check5==1:
  RecruitmentDF = Rec.join(joined_df,[Rec["Request_Id"] == joined_df["Request_Id"]] , how = 'inner' ).drop(joined_df["Request_Id"])
  #RecruitmentDF.show()

+----------+--------------------+---------------+------------+-----------------+--------------------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+
Request_Id| Role| Experience| Client|Total_Requisition| Project| Opened_On|Recruiter|Priority| Location|On_Hold_Date|Resumed_Date|Cancelled_Date|Skills|Role_Category|Status|Remarks|joined_cnt|
+----------+--------------------+---------------+------------+-----------------+--------------------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+
 1|Solution Architec...| Mid level| Timken| 4| Timken|2019-02-01 00:00:00| Raja| P1|Bangalore| null| null| null| null| developer| null| null| 2|
 2| QA-Automation| Lead level| GS| 7| GS|2019-02-01 00:00:00| Babji| P1|Bangalore| null| null| null| null| developer| null| null| 1|
 3| .NET developers| Freshers| RRD| 5| Logistics|2019-02-04 00:00:00| Poornima| P2|Bangalore| null| null| null| null| developer| null| null| 1|
 5|Test Engineer(bac...| Entry level| Timken| 2| Timken|2019-02-07 00:00:00| Poornima| P1|Bangalore| null| null| null| null| Tester| null| null| 1|
 6| Core Java-Senior|Executive level| GS| 1| GS-Victor|2019-02-08 00:00:00| Karthik| P1|Bangalore| null| null| null| null| developer| null| null| 1|
 7| Java8/SPARK/HIVE|Associate level| TechM| 1| TechM|2019-02-08 00:00:00| Khushboo| P2|Bangalore| null| null| null| null| developer| null| null| 1|
 8|Senior Java Devel...| Lead level| RRD| 1| Logistics|2019-02-13 00:00:00| Poornima| P1|Bangalore| null| null| null| null| developer| null| null| 1|
 9| Architects| Lead level| TechM-BT| 6| BT|2019-02-13 00:00:00| Lavanya| P1|Bangalore| null| null| null| null| developer| null| null| 1|
 10| Freshers(1QA+5.NET)|Associate level| RRD| 5|RRD ( Internation...|2019-02-14 00:00:00| Raja| P1|Bangalore| null| null| null| null| developer| null| null| 3|
 11|Full stack-Analyt...| Freshers| GS| 5|Workflow Engineering|2019-02-18 00:00:00| Bhavana| P1|Bangalore| null| null| null| null| developer| null| null| 1|
 12|Solution Architec...| Lead level| GS| 1| Mazhaar|2019-02-19 00:00:00| Karthik| P2|Bangalore| null| null| null| null| developer| null| null| 1|
 13|LTE Automation Te...|Executive level| RRD| 9|RRD ( Internation...|2019-02-20 00:00:00| Raja| P1|Bangalore| null| null| null| null| Tester| null| null| 6|
 14| Python| Freshers| LSC| 3| LSC|2019-02-20 00:00:00| Poornima| P1| Chennai| null| null| null| null| developer| null| null| 2|
 16| .NET Lead| Freshers|Messagepoint| 7| Messagepoint|2019-02-22 00:00:00| Poornima| P2| India| null| null| null| null| developer| null| null| 1|
 17| Big Data| Top level| TechM-BT| 4| BT|2019-02-25 00:00:00| Vignesh| P2|Bangalore| null| null| null| null| developer| null| null| 2|
 18|Unix/Perl Scripti...| Mid level| TechM| 5| TechM|2019-02-25 00:00:00| Babji| P2|Bangalore| null| null| null| null| developer| null| null| 4|
 19| Spring Hibernate| Entry level| GS| 3|Mahesh Sharma/Equ...|2019-02-26 00:00:00| Karthik| P1| Chennai| null| null| null| null| developer| null| null| 3|
 20| PL/SQL Developer| Senior level| RRD| 8| RRD (Logistics)|2019-02-27 00:00:00| Raja| P2|Bangalore| null| null| null| null| developer| null| null| 8|
 21|Java Engineer (On...| Entry level| TechM-BT| 5| BT|2019-03-01 00:00:00| Umair| P2|Bangalore| null| null| null| null| developer| null| null| 3|
 22| Core Java - Lead| Top level| ArtVan| 9| Saggezza|2019-03-01 00:00:00| Babji| P2|Bangalore| null| null| null| null| developer| null| null| 3|
+----------+--------------------+---------------+------------+-----------------+--------------------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+
only showing top 20 rows

In [13]:
# Number of positions yet to be filled.
if check1==1&check2==1&check3==1&check4==1&check5==1:
  RecruitDF=RecruitmentDF.withColumn("Balance_Position",RecruitmentDF["Total_Requisition"]-RecruitmentDF["joined_cnt"])
  #RecruitDF.show(2)

+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+
Request_Id| Role|Experience|Client|Total_Requisition|Project| Opened_On|Recruiter|Priority| Location|On_Hold_Date|Resumed_Date|Cancelled_Date|Skills|Role_Category|Status|Remarks|joined_cnt|Balance Position|
+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+
 1|Solution Architec...| Mid level|Timken| 4| Timken|2019-02-01 00:00:00| Raja| P1|Bangalore| null| null| null| null| developer| null| null| 2| 2|
 2| QA-Automation|Lead level| GS| 7| GS|2019-02-01 00:00:00| Babji| P1|Bangalore| null| null| null| null| developer| null| null| 1| 6|
+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+
only showing top 2 rows

In [14]:
# Add a new column status to the recruitment dataframe and fill it with open or closed depending on the requirements.
if check1==1&check2==1&check3==1&check4==1&check5==1:
  RecruitDF=RecruitDF.withColumn('Status',when(RecruitDF["Balance_Position"]>0,"Open" ).otherwise("Closed"))
#   RecruitDF=RecruitDF.withColumn('Status',when(col("On_Hold_Date").isNotNull() & col("Resumed_Date").isNull(),"Open" ))
 # RecruitDF.show(1)

+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+
Request_Id| Role|Experience|Client|Total_Requisition|Project| Opened_On|Recruiter|Priority| Location|On_Hold_Date|Resumed_Date|Cancelled_Date|Skills|Role_Category|Status|Remarks|joined_cnt|Balance Position|
+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+
 1|Solution Architec...| Mid level|Timken| 4| Timken|2019-02-01 00:00:00| Raja| P1|Bangalore| null| null| null| null| developer| Open| null| 2| 2|
+----------+--------------------+----------+------+-----------------+-------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+
only showing top 1 row

In [15]:
# RecruitDF=RecruitDF.withColumn('Status',when((col("Balance Position")>0) & (col('On_Hold_Date').isNull()),"Open" ).otherwise(when((col("On_Hold_Date").isNotNull() )& (col("Resumed_Date").isNull()),"oh-hold" ).otherwise(when((col("On_Hold_Date").isNotNull()) & (col("Resumed_Date").isNotNull()) & (col("Balance Position")>0,"Closed"))).otherwise('open')))
# #.when(col("On_Hold_Date").isNotNull() & col("Resumed_Date").isNull(),"oh-hold" )
# #RecruitDF=RecruitDF.withColumn('Status',when(col("On_Hold_Date").isNotNull() & col("Resumed_Date").isNull(),"Open" ))
# RecruitDF.show(3)

In [16]:
# # Total number of recruits joined corresponding to the recruiter and Role.
# if check1==1&check2==1&check3==1&check4==1&check5==1:
#   EmpDF= Emp.join(RecruitDF["Request_Id","Recriuter"],[Emp["Request_Id"] == RecruitDF["Request_Id"]] , how ='inner').drop(Emp["Request_Id"])
#   EmpDF.show()

In [17]:
# #Adding opening date to the employee dataframe.
# if check1==1&check2==1&check3==1&check4==1&check5==1:
#   EmpDF= EmpDF.join(RecruitDF["Request_Id","Opened_On"],[EmpDF["Request_Id"] == RecruitDF["Request_Id"]] , how ='inner').drop(EmpDF["Request_Id"])
#   EmpDF.show()

In [18]:
# Ageing time calculation. i.e the diff in time between current date and requirement open date.
if check1==1&check2==1&check3==1&check4==1&check5==1:
  RecruitDF1=RecruitDF.withColumn("Ageing",datediff(col("Resumed_Date"), col("On_Hold_Date")))
  RecruitDF=RecruitDF1.withColumn("Ageing", when(RecruitDF1.Ageing.isNull(), lit(0)).otherwise(RecruitDF1.Ageing))
  RecruitDF=RecruitDF.withColumn("Ageing1",datediff(current_timestamp(), col("Opened_On")))
  RecruitDF=RecruitDF.withColumn("NetAgeing",col("Ageing1")-col("Ageing")).drop(RecruitDF["Ageing1"])
 # RecruitDF.show(5)
 


+----------+--------------------+---------------+------+-----------------+---------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+------+---------+
Request_Id| Role| Experience|Client|Total_Requisition| Project| Opened_On|Recruiter|Priority| Location|On_Hold_Date|Resumed_Date|Cancelled_Date|Skills|Role_Category|Status|Remarks|joined_cnt|Balance Position|Ageing|NetAgeing|
+----------+--------------------+---------------+------+-----------------+---------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+------+---------+
 1|Solution Architec...| Mid level|Timken| 4| Timken|2019-02-01 00:00:00| Raja| P1|Bangalore| null| null| null| null| developer| Open| null| 2| 2| 0| 306|
 2| QA-Automation| Lead level| GS| 7| GS|2019-02-01 00:00:00| Babji| P1|Bangalore| null| null| null| null| developer| Open| null| 1| 6| 0| 306|
 3| .NET developers| Freshers| RRD| 5|Logistics|2019-02-04 00:00:00| Poornima| P2|Bangalore| null| null| null| null| developer| Open| null| 1| 4| 0| 303|
 5|Test Engineer(bac...| Entry level|Timken| 2| Timken|2019-02-07 00:00:00| Poornima| P1|Bangalore| null| null| null| null| Tester| Open| null| 1| 1| 0| 300|
 6| Core Java-Senior|Executive level| GS| 1|GS-Victor|2019-02-08 00:00:00| Karthik| P1|Bangalore| null| null| null| null| developer|Closed| null| 1| 0| 0| 299|
+----------+--------------------+---------------+------+-----------------+---------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+------+---------+
only showing top 5 rows

In [19]:
# Hire Time calculation .i.e the time difference between joining date and requirement open date. 
if check1==1&check2==1&check3==1&check4==1&check5==1:
  R1 = Rec[["Request_Id","On_Hold_Date","Resumed_Date",'Project','Client','Opened_On','Recruiter']]
  EmpDF= Emp.join(R1,[Emp["Request_Id"] == R1["Request_Id"]] , how ='inner').drop(Emp["Request_Id"]) 
  EmpDF=EmpDF.withColumn('Hire_Time', F.when(F.col('Joined_Date')>F.col('Resumed_Date'),datediff(col("Resumed_Date"), col("Joined_Date")))                                    .otherwise(datediff(col("Joined_Date"), col("Opened_On")))).drop(EmpDF.Resumed_Date).drop(EmpDF.On_Hold_Date)
  #EmpDF.show()

+------+----+------------+-------------------+-------+-----------+---------+----------+--------------------+--------+-------------------+---------+---------+
Emp_Id|Name|Candidate_Id| Joined_Date| Salary|Last_Salary|Hike_Pert|Request_Id| Project| Client| Opened_On|Recruiter|Hire Time|
+------+----+------------+-------------------+-------+-----------+---------+----------+--------------------+--------+-------------------+---------+---------+
 1|null| 5|2019-02-12 00:00:00| 400000| 285714| 40.0| 3| Logistics| RRD|2019-02-04 00:00:00| Poornima| 8|
 2|null| 6|2019-02-20 00:00:00| 850000| 500000| 70.0| 10|RRD ( Internation...| RRD|2019-02-14 00:00:00| Raja| 6|
 4|null| 12|2019-02-27 00:00:00|1500000| 937500| 60.0| 12| Mazhaar| GS|2019-02-19 00:00:00| Karthik| 8|
 5|null| 13|2019-03-01 00:00:00| 950000| 527778| 80.0| 10|RRD ( Internation...| RRD|2019-02-14 00:00:00| Raja| 15|
 6|null| 16|2019-03-11 00:00:00|3550000| 2535714| 40.0| 17| BT|TechM-BT|2019-02-25 00:00:00| Vignesh| 14|
 7|null| 18|2019-03-13 00:00:00|7050000| 3710526| 90.0| 22| Saggezza| ArtVan|2019-03-01 00:00:00| Babji| 12|
 8|null| 21|2019-03-14 00:00:00| 350000| 250000| 40.0| 14| LSC| LSC|2019-02-20 00:00:00| Poornima| 22|
 9|null| 22|2019-03-15 00:00:00|3450000| 2653846| 30.0| 26| BT|TechM-BT|2019-03-14 00:00:00| Umair| 1|
 11|null| 24|2019-03-21 00:00:00|1000000| 714286| 40.0| 10|RRD ( Internation...| RRD|2019-02-14 00:00:00| Raja| 35|
 12|null| 25|2019-03-22 00:00:00|1350000| 794118| 70.0| 18| TechM| TechM|2019-02-25 00:00:00| Babji| 25|
 13|null| 26|2019-03-26 00:00:00|3250000| 2321429| 40.0| 13|RRD ( Internation...| RRD|2019-02-20 00:00:00| Raja| 34|
 14|null| 27|2019-03-27 00:00:00|1650000| 1031250| 60.0| 36| Michael/Vikas| Tigo|2019-03-27 00:00:00| Sonali| 0|
 15|null| 28|2019-04-05 00:00:00|1450000| 1115385| 30.0| 36| Michael/Vikas| Tigo|2019-03-27 00:00:00| Sonali| 9|
 16|null| 31|2019-04-08 00:00:00| 400000| 285714| 40.0| 14| LSC| LSC|2019-02-20 00:00:00| Poornima| 47|
 17|null| 33|2019-04-10 00:00:00| 950000| 593750| 60.0| 7| TechM| TechM|2019-02-08 00:00:00| Khushboo| 61|
 18|null| 36|2019-04-10 00:00:00| 600000| 375000| 60.0| 19|Mahesh Sharma/Equ...| GS|2019-02-26 00:00:00| Karthik| 43|
 19|null| 38|2019-04-11 00:00:00|1800000| 1058824| 70.0| 24| TechM| TechM|2019-03-12 00:00:00| Khushboo| 30|
 20|null| 42|2019-04-11 00:00:00|3350000| 1970588| 70.0| 26| BT|TechM-BT|2019-03-14 00:00:00| Umair| 28|
 21|null| 44|2019-04-15 00:00:00|1450000| 1035714| 40.0| 36| Michael/Vikas| Tigo|2019-03-27 00:00:00| Sonali| 19|
 23|null| 50|2019-04-17 00:00:00|6900000| 5307692| 30.0| 22| Saggezza| ArtVan|2019-03-01 00:00:00| Babji| 47|
+------+----+------------+-------------------+-------+-----------+---------+----------+--------------------+--------+-------------------+---------+---------+
only showing top 20 rows

In [21]:
#Conversion rate calculation
if check1==1&check2==1&check3==1&check4==1&check5==1:
  RecruitDF=RecruitDF.withColumn("ConversionRate",F.round((RecruitDF["joined_cnt"]/RecruitDF["Total_Requisition"])*100,2)).drop(RecruitDF["Ageing"])
 # RecruitDF.show(3)

+----------+--------------------+----------+------+-----------------+---------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+---------+--------------+
Request_Id| Role|Experience|Client|Total_Requisition| Project| Opened_On|Recruiter|Priority| Location|On_Hold_Date|Resumed_Date|Cancelled_Date|Skills|Role_Category|Status|Remarks|joined_cnt|Balance Position|NetAgeing|ConversionRate|
+----------+--------------------+----------+------+-----------------+---------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+---------+--------------+
 1|Solution Architec...| Mid level|Timken| 4| Timken|2019-02-01 00:00:00| Raja| P1|Bangalore| null| null| null| null| developer| Open| null| 2| 2| 306| 50.0|
 2| QA-Automation|Lead level| GS| 7| GS|2019-02-01 00:00:00| Babji| P1|Bangalore| null| null| null| null| developer| Open| null| 1| 6| 306| 14.29|
 3| .NET developers| Freshers| RRD| 5|Logistics|2019-02-04 00:00:00| Poornima| P2|Bangalore| null| null| null| null| developer| Open| null| 1| 4| 303| 20.0|
+----------+--------------------+----------+------+-----------------+---------+-------------------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+---------+--------------+
only showing top 3 rows

In [22]:
 import datetime
if check1==1&check2==1&check3==1&check4==1&check5==1:
  dat1=datetime.datetime.now()

In [23]:
from pyspark.sql.types import DateType
RecruitDF= RecruitDF.withColumn("Opened_On",RecruitDF['Opened_On'].cast(DateType()))
RecruitDF= RecruitDF.withColumn("On_Hold_Date",RecruitDF['On_Hold_Date'].cast(DateType()))
RecruitDF= RecruitDF.withColumn("Resumed_Date",RecruitDF['Resumed_Date'].cast(DateType()))
EmpDF= EmpDF.withColumn("Opened_On",EmpDF['Opened_On'].cast(DateType()))


In [24]:
#ReferralDF.show()

+-----------+------------+------+-------------+--------+
Referral_Id|Candidate_Id|Emp_Id|Referral_Date| Status|
+-----------+------------+------+-------------+--------+
 196| 1261| 459| null|selected|
 195| 1258| 458| null| ongoing|
 192| 1251| 455| null|selected|
 198| 1265| 460| null|selected|
 194| 1254| 457| null|selected|
 197| 1264| 460| null| ongoing|
 191| 1250| 454| null| ongoing|
 193| 1253| 456| null| ongoing|
 190| 1248| 453| null| ongoing|
 189| 1266| 47| null|selected|
+-----------+------------+------+-------------+--------+

In [25]:
if check1==1&check2==1&check3==1&check4==1&check5==1:
  try:
    RecruitDF.repartition(1).write.csv("abfss://result1@recruitstg.dfs.core.windows.net/Recruitment/Recruitment_" + str(dat1),header = True)
    EmpDF.repartition(1).write.csv("abfss://result1@recruitstg.dfs.core.windows.net/Employee/Employee_" + str(dat1),header = True)
    CandidateDF.repartition(1).write.csv("abfss://result1@recruitstg.dfs.core.windows.net/Candidate/Candidate_" + str(dat1),header = True)
    ReferralDF.repartition(1).write.csv("abfss://result1@recruitstg.dfs.core.windows.net/Referral/Referral_" + str(dat1),header = True)
    interviewDF.repartition(1).write.csv("abfss://result1@recruitstg.dfs.core.windows.net/interview/interview_" + str(dat1),header = True)
  except Exception as e:
    print(e)

In [26]:
interviewDF.show(1)

+------------+------------+--------------+---------+------+--------------+-------+-------+
Interview_Id|Candidate_Id|Interviewer_Id| Round|Result|Interview_Date| Status|Remarks|
+------------+------------+--------------+---------+------+--------------+-------+-------+
 1| 1261| null|paper&pen|passed| null|ongoing| null|
+------------+------------+--------------+---------+------+--------------+-------+-------+
only showing top 1 row

In [27]:
#RecruitDF.show(1)

+----------+--------------------+----------+------+-----------------+-------+----------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+---------+--------------+
Request_Id| Role|Experience|Client|Total_Requisition|Project| Opened_On|Recruiter|Priority| Location|On_Hold_Date|Resumed_Date|Cancelled_Date|Skills|Role_Category|Status|Remarks|joined_cnt|Balance Position|NetAgeing|ConversionRate|
+----------+--------------------+----------+------+-----------------+-------+----------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+---------+--------------+
 1|Solution Architec...| Mid level|Timken| 4| Timken|2019-02-01| Raja| P1|Bangalore| null| null| null| null| developer| Open| null| 2| 2| 306| 50.0|
+----------+--------------------+----------+------+-----------------+-------+----------+---------+--------+---------+------------+------------+--------------+------+-------------+------+-------+----------+----------------+---------+--------------+
only showing top 1 row

In [28]:
#EmpDF.show(1)

+------+----+------------+-------------------+------+-----------+---------+----------+---------+------+----------+---------+---------+
Emp_Id|Name|Candidate_Id| Joined_Date|Salary|Last_Salary|Hike_Pert|Request_Id| Project|Client| Opened_On|Recruiter|Hire Time|
+------+----+------------+-------------------+------+-----------+---------+----------+---------+------+----------+---------+---------+
 1|null| 5|2019-02-12 00:00:00|400000| 285714| 40.0| 3|Logistics| RRD|2019-02-04| Poornima| 8|
+------+----+------------+-------------------+------+-----------+---------+----------+---------+------+----------+---------+---------+
only showing top 1 row

In [29]:
#CandidateDF.show(1)

+----+------+--------+------------+--------+-----------------+-----------+-------------------+-------+-----------------+------+-------+-------+------------+
Name|Gender|Email_Id|Phone_Number| Source|Career_Started_At|Break_Taken|Relevant_Experience|Offered| Round|Result| Status|Remarks|Candidate_Id|
+----+------+--------+------------+--------+-----------------+-----------+-------------------+-------+-----------------+------+-------+-------+------------+
null| F| null| null|referral| null| null| 0.0| null|technical round-2|passed|ongoing| null| 1246|
+----+------+--------+------------+--------+-----------------+-----------+-------------------+-------+-----------------+------+-------+-------+------------+
only showing top 1 row

In [30]:
#ReferralDF.show(1)

+-----------+------------+------+-------------+--------+
Referral_Id|Candidate_Id|Emp_Id|Referral_Date| Status|
+-----------+------------+------+-------------+--------+
 196| 1261| 459| null|selected|
+-----------+------------+------+-------------+--------+
only showing top 1 row